In [104]:
import pandas as pd
# from sparql import query, display_result
from SPARQLWrapper import SPARQLWrapper, CSV, JSON
from io import BytesIO

In [105]:
sparql = SPARQLWrapper(
    "https://fedlex.data.admin.ch/sparqlendpoint"
)
sparql.setReturnFormat(CSV)

# Get all currently applicable laws in DEU in XML-format
sparql.setQuery("""
    PREFIX jolux: <http://data.legilux.public.lu/resource/ontology/jolux#>
    PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
    PREFIX lang: <http://publications.europa.eu/resource/authority/language/>

    select distinct  (str(?srNotation) as ?rsNr) (str(?dateApplicability) as ?dateApplicability) ?title ?abrev ?consoExpr ?fileUrl {
    filter(?language = lang:DEU)
    #FRA = français, ITA = italiano, DEU = deutsch, ENG = english, ROH = rumantsch
    ?consolidation a jolux:Consolidation .
    ?consolidation jolux:dateApplicability ?dateApplicability .
    optional { ?consolidation jolux:dateEndApplicability ?dateEndApplicability }
    filter(xsd:date(?dateApplicability) <= xsd:date(now()) && (!bound(?dateEndApplicability) || xsd:date(?dateEndApplicability) >= xsd:date(now())))
    ?consolidation jolux:isRealizedBy ?consoExpr .
    ?consoExpr jolux:language ?language .
    ?consoExpr jolux:isEmbodiedBy ?consoManif .
    ?consoManif jolux:userFormat <https://fedlex.data.admin.ch/vocabulary/user-format/xml> .
    #for pdf-files: replace doc with pdf-a
    #for html-files: replace doc with html
    #for xml-files: replace doc with xml
    ?consoManif jolux:isExemplifiedBy ?fileUrl .
    ?consolidation jolux:isMemberOf ?cc .
    ?cc jolux:classifiedByTaxonomyEntry/skos:notation ?srNotation .
    optional { ?cc jolux:dateNoLongerInForce ?ccNoLonger }
    optional { ?cc jolux:dateEndApplicability ?ccEnd }
    filter(!bound(?ccNoLonger) || xsd:date(?ccNoLonger) > xsd:date(now()))
    filter(!bound(?ccEnd) || xsd:date(?ccEnd) >= xsd:date(now()))
    filter(datatype(?srNotation) = <https://fedlex.data.admin.ch/vocabulary/notation-type/id-systematique>)
    optional {
        ?cc jolux:isRealizedBy ?ccExpr .
        ?ccExpr jolux:language ?language .
        ?ccExpr jolux:title ?title .
        optional {?ccExpr jolux:titleShort ?abrev }
    }
    }
    order by ?srNotation
    """
)

try:
    ret = sparql.queryAndConvert()
    # df = pd.read_csv(ret.decode("utf-8"))
    df = pd.read_csv(BytesIO(ret))
    # for r in ret["results"]["bindings"]:
    #     print(r)
except Exception as e:
    print(e)

In [106]:
import os
import requests

In [107]:
def download_file(url, file_path):
    # Senden einer GET-Anfrage an die Website
    response = requests.get(url)

    # Überprüfen des Statuscodes
    if response.status_code == 200:
        # Die Website antwortete erfolgreich
        # Das XML-Dokument in einer Datei speichern
        with open(file_path, "wb") as file:
            file.write((response.content))

        print("Das Dokument wurde erfolgreich heruntergeladen.")
    else:
        # Die Website antwortete mit einem Fehler
        print(f"Fehler beim Herunterladen des Dokuments: {response.status_code}")

In [109]:
file_folder = "files_xml"

if not os.path.exists(file_folder):
    os.makedirs(file_folder)


download_rsNr = ['822.111', '822.112', '822.113', '822.114', '822.115']

# Download all respective files
for rsNr in download_rsNr:
    fileUrl = df['fileUrl'][df['rsNr'] == rsNr].item()
    print('d', fileUrl)
    file_path = os.path.join(file_folder, rsNr + '.xml')
    download_file(fileUrl, file_path)


d https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/2000/243/20230901/de/xml/fedlex-data-admin-ch-eli-cc-2000-243-20230901-de-xml.xml
Das Dokument wurde erfolgreich heruntergeladen.
d https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/2000/244/20231015/de/xml/fedlex-data-admin-ch-eli-cc-2000-244-20231015-de-xml-1.xml
Das Dokument wurde erfolgreich heruntergeladen.
d https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1993/2553_2553_2553/20151001/de/xml/fedlex-data-admin-ch-eli-cc-1993-2553_2553_2553-20151001-de-xml-3.xml
Das Dokument wurde erfolgreich heruntergeladen.
d https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/1993/2564_2564_2564/20150501/de/xml/fedlex-data-admin-ch-eli-cc-1993-2564_2564_2564-20150501-de-xml-3.xml
Das Dokument wurde erfolgreich heruntergeladen.
d https://fedlex.data.admin.ch/filestore/fedlex.data.admin.ch/eli/cc/2007/692/20180701/de/xml/fedlex-data-admin-ch-eli-cc-2007-692-20180701-de-xml-4.xml
D